In [1]:
# imports
import os

import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, recall_score

In [2]:
# configs
PROJECT_PATH = "/home/alex/paper-2025-anonymous-submission"

In [3]:
# load data

dataset = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_data/dataset.json"
    ),
    orient="index"
)

dev_dataset = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/processed_data/dev_dataset.json"
    ),
    orient="index"
)

/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/home/alex/aspirantura_hse/general_venv/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [4]:
df = pd.concat([dataset, dev_dataset])

In [5]:
df

,annotations,summary,is_word_play,date,article_url,headline,lead
0,None,Жилищный нацпроект помимо количественных целей...,False,2021-12-16,https://www.kommersant.ru/doc/5130217,Комфортная среда,Городскую среду улучшают с учетом мнения жителей
1,"[{'headline_substring': 'стушевалась', 'start_...",Окончание летних каникул и сезона шашлыков при...,True,2021-08-09,https://www.kommersant.ru/doc/4976692,Свинина стушевалась,Началось сезонное снижение цен
2,"[{'headline_substring': '«Истока»', 'start_ind...","Как стало известно “Ъ”, главное военное следст...",True,2023-09-11,https://www.kommersant.ru/doc/6210688,Исток с «Истока»,Раскрыта преступная группа несунов с оборонног...
3,"[{'headline_substring': 'Налетай, позеленело',...","Как выяснил “Ъ”, крупнейший инвестор в солнечн...",True,2021-04-08,https://www.kommersant.ru/doc/4928249,"Налетай, позеленело",Компания «Хевел» выставила электроэнергию со с...
4,[{'headline_substring': 'Гриппы по осени счита...,В России стартовала кампания прививок против г...,True,2023-09-11,https://www.kommersant.ru/doc/6210686,Гриппы по осени считают,В России началась массовая вакцинация от вируса
...,...,...,...,...,...,...,...
195,"[{'headline_substring': 'подошли', 'start_inde...",Основными покупателями стартапов в 2021 году с...,True,2021-08-12,https://www.kommersant.ru/doc/5117889,К стартапам подошли экосистемно,Корпорации начали активнее инвестировать в нов...
196,[{'headline_substring': 'Звезда по имени Моди'...,Новость об успешной посадке на Луне индийской ...,True,2023-08-24,https://www.kommersant.ru/doc/6174473,Звезда по имени Моди,Обозреватель “Ъ” Сергей Строкань — о новом гло...
197,None,"В Кемерово суд арестовал Михаила Федяева, през...",False,2021-12-15,https://www.kommersant.ru/doc/5130733,Собственник подорвался на шахте,По делу о ЧП на «Листвяжной» арестован президе...
198,None,Авторынок РФ в августе продолжил медленное вос...,False,2022-08-30,https://www.kommersant.ru/doc/5535961,Авторынок растет на льготных кредитах,Lada обеспечила более трети продаж в августе


In [6]:
def extract_annotations_labels(annotations):
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict):
                all_wordplay_type.append(
                    annot["wordplay_type"]
                )
            elif isinstance(annot, list):
                all_wordplay_type.append(
                    annot[0]["wordplay_type"]
                )
    
    return all_wordplay_type


In [7]:
def extract_annotations_ref_str(annotations):
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict):
                all_wordplay_type.append(
                    annot["reference_string"]
                )
            elif isinstance(annot, list):
                for t_ann in annot:
                    all_wordplay_type.append(
                        t_ann["reference_string"]
                    )
    
    return all_wordplay_type


In [8]:
def extract_annotations_headline_substring(annotations):
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict):
                all_wordplay_type.append(
                    annot["headline_substring"]
                )
            elif isinstance(annot, list):
                for t_ann in annot:
                    all_wordplay_type.append(
                        t_ann["headline_substring"]
                    )
    
    return all_wordplay_type

In [9]:
wikipedia_urls = 0
witionary_urls = 0

In [10]:
def extract_annotations_labels_with_links(annotations):

    global wikipedia_urls
    global witionary_urls
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict) and "http" in annot["reference_url"]:
                all_wordplay_type.append(
                    annot["wordplay_type"]
                )
                if annot["wordplay_type"] in ["Collocation", "Reference", "Idiom"]:
                    if "wikipedia" in annot["reference_url"]:
                        wikipedia_urls += 1
                    else:
                        witionary_urls += 1
            elif isinstance(annot, list) and "http" in annot[0]["reference_url"]:
                all_wordplay_type.append(
                    annot[0]["wordplay_type"]
                )

                if annot[0]["wordplay_type"] in ["Collocation", "Reference", "Idiom"]:
                    if "wikipedia" in annot[0]["reference_url"]:
                        wikipedia_urls += 1
                    else:
                        witionary_urls += 1
    
    return all_wordplay_type


In [11]:
df["all_wordplay_type"] = df["annotations"].apply(extract_annotations_labels)


In [12]:
wikipedia_urls, witionary_urls

(0, 0)

In [13]:
df["all_wordplay_type_link"] = df["annotations"].apply(extract_annotations_labels_with_links)

In [14]:
df["all_wordplay_type"].explode().value_counts()

Collocation            424
Reference              352
Polysemy               190
Nonce word             185
Idiom                  177
Phonetic similarity     98
Oxymoron                48
Homonymy                26
Name: all_wordplay_type, dtype: int64

In [15]:
df["all_wordplay_type_link"].explode().value_counts()

Reference              213
Collocation            127
Idiom                  118
Homonymy                 4
Polysemy                 2
Nonce word               2
Phonetic similarity      2
Name: all_wordplay_type_link, dtype: int64

In [16]:
def extract_links(annotations):
    
    all_wordplay_type = list()
    
    if isinstance(annotations, list):
        for annot in annotations:
            if isinstance(annot, dict) and "http" in annot["reference_url"]:
                all_wordplay_type.append(
                    annot["reference_url"]
                )
            elif isinstance(annot, list) and "http" in annot[0]["reference_url"]:
                all_wordplay_type.append(
                    annot[0]["reference_url"]
                )
    
    return all_wordplay_type


In [17]:
df["all_links"] = df["annotations"].apply(extract_links)

In [18]:
len(df["all_links"].explode().dropna().tolist())

468

In [19]:
df["all_reference_string"] = df["annotations"].apply(extract_annotations_ref_str)

In [20]:
df[["all_reference_string"]]

,all_reference_string
0,[]
1,[]
2,[]
3,[налетай - подешевело]
4,[цыплят по осени считают]
...,...
195,"[подошли системно, [системно, экосистема]]"
196,[Звезда по имени Солнце]
197,[]
198,[]


In [21]:
df["all_headline_substring"] = df["annotations"].apply(extract_annotations_headline_substring)

In [22]:
df["all_reference_string"].explode().dropna().apply(len).median()

15.0

In [23]:
df["all_reference_string"].explode().explode().dropna().apply(lambda x: x.split(" ")).apply(len).median()

2.0

In [24]:
set(df["all_wordplay_type"].explode().tolist())

{'Collocation',
 'Homonymy',
 'Idiom',
 'Nonce word',
 'Oxymoron',
 'Phonetic similarity',
 'Polysemy',
 'Reference',
 nan}

In [25]:
stats_tbl = dict()

In [26]:
all_types = [
 'Polysemy',
 'Homonymy',
 'Phonetic similarity',
 'Collocation',
 'Idiom',
 'Reference',
 'Nonce word',
 'Oxymoron',
 ]

for i, tt in enumerate(all_types):
    tmp_df = df.copy()
    tmp_df["is_need"] = tmp_df["all_wordplay_type"].apply(lambda x: True if tt in x else False)
    tmp_df = tmp_df[tmp_df["is_need"]]

    stats_tbl[i] = {
        "wordplay type": tt,
        "word len mean": np.round(tmp_df["all_headline_substring"].explode().dropna().apply(lambda x: x.split(" ")).apply(len).mean(), 2),
        "symbol len mean": np.round(tmp_df["all_headline_substring"].explode().dropna().apply(len).mean(), 2)
    }

    print(tt, np.round(tmp_df["all_headline_substring"].explode().dropna().apply(len).mean(), 2), np.round(tmp_df["all_headline_substring"].explode().dropna().apply(lambda x: x.split(" ")).apply(len).mean(), 2))
    

Polysemy 10.45 1.51
Homonymy 8.29 1.57
Phonetic similarity 10.47 1.8
Collocation 18.44 2.64
Idiom 20.18 3.43
Reference 22.93 3.73
Nonce word 13.76 1.44
Oxymoron 16.18 2.02


In [27]:
print(pd.DataFrame().from_dict(stats_tbl, orient="index").to_latex())

\begin{tabular}{llrr}
\toprule
{} &        wordplay type &  word len mean &  symbol len mean \\
\midrule
0 &             Polysemy &           1.51 &            10.45 \\
1 &             Homonymy &           1.57 &             8.29 \\
2 &  Phonetic similarity &           1.80 &            10.47 \\
3 &          Collocation &           2.64 &            18.44 \\
4 &                Idiom &           3.43 &            20.18 \\
5 &            Reference &           3.73 &            22.93 \\
6 &           Nonce word &           1.44 &            13.76 \\
7 &             Oxymoron &           2.02 &            16.18 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1751/1386570770.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.DataFrame().from_dict(stats_tbl, orient="index").to_latex())


In [29]:
df["all_wordplay_type_len"] = df["all_wordplay_type"].apply(lambda x: len(x))

In [66]:
df[
    df["all_wordplay_type_len"] > 1
].sample(2)

,annotations,summary,is_word_play,date,article_url,headline,lead,all_wordplay_type,all_wordplay_type_link,all_links,all_reference_string,all_headline_substring,all_wordplay_type_len
253,[{'headline_substring': 'Диалектический пилома...,Пиломатериалы и лесопромышленная продукция нач...,True,2021-10-27,https://www.kommersant.ru/doc/5051268,Диалектический пиломатериализм,Цены на фанеру и доски начали снижаться вслед ...,"[Collocation, Nonce word]",[Collocation],[https://ru.wikipedia.org/wiki/Диалектический_...,"[Диалектический материализм, [материализм, пил...","[Диалектический пиломатериализм, пиломатериализм]",2
2200,"[{'headline_substring': 'Чистые', 'start_index...",На фоне участившихся заявлений компаний об амб...,True,2021-10-08,https://www.kommersant.ru/doc/4910877,Чистые нулевые выбросы нечисты,Экологи критикуют планы компаний по достижению...,"[Oxymoron, Oxymoron]",[],[],"[, ]","[Чистые, нечисты]",2


In [52]:
from pprint import pprint

In [59]:
"https://www.kommersant.ru/doc/6212002"

'https://www.kommersant.ru/doc/6212002'

In [67]:
df[df["article_url"] == "https://www.kommersant.ru/doc/5051268"]["annotations"].iloc[0]

[{'headline_substring': 'Диалектический пиломатериализм',
  'start_index': 0,
  'end_index': 30,
  'wordplay_type': 'Collocation',
  'reference_string': 'Диалектический материализм',
  'reference_url': 'https://ru.wikipedia.org/wiki/Диалектический_материализм'},
 {'headline_substring': 'пиломатериализм',
  'start_index': 15,
  'end_index': 30,
  'wordplay_type': 'Nonce word',
  'reference_string': ['материализм', 'пиломатериалы'],
  'reference_url': ['', '']}]

In [35]:
df["all_headline_substring"].explode().dropna().apply(len).mean()

17.616810059563203

In [36]:
np.round(df["all_headline_substring"].explode().dropna().apply(lambda x: x.split(" ")).apply(len).mean(), 2)

2.65

In [62]:
df[["headline", "all_wordplay_type"]].sample(3)

,headline,all_wordplay_type
2343,Иноагент остался без прецедента,[]
64,Батареи просят рубля,[]
260,Штамм «дельта» пробил Великую иммунную стену,[Collocation]


In [63]:
df["all_wordplay_type_len"] = df["all_wordplay_type"].apply(lambda x: len(x))

In [64]:
df["all_wordplay_type_len_set"] = df["all_wordplay_type"].apply(lambda x: len(set(x)))

In [66]:
len(df[df["all_wordplay_type_len"] != df["all_wordplay_type_len_set"]])

29